In [3]:
%matplotlib inline
import pandas as pd
import os
import matplotlib as plt
import seaborn as sns
import numpy as np
import statsmodels.formula.api as sm

In [4]:
#custom style HTML output

from IPython.core.display import HTML

csspath1 = r'C:\COURSERA\PYCON2015_TUTORIALS\Brandon Rhodes - Pandas From The Ground Up - PyCon 2015\style-table.css'
csspath2 = r'C:\COURSERA\PYCON2015_TUTORIALS\Brandon Rhodes - Pandas From The Ground Up - PyCon 2015\style-notebook.css'

css = open(csspath1).read() + open(csspath2).read()
HTML('<style>{}</style>'.format(css))

In [5]:
# Load the training and testing sets and 
# save them as variables with the names pisaTrain and pisaTest.

pisaTest = pd.read_csv('DATA\pisa2009test.csv')
pisaTrain = pd.read_csv('DATA\pisa2009train.csv')

In [6]:
# How many students are there in the training set?

pisaTrain.shape , pisaTest.shape

((3663, 24), (1570, 24))

In [7]:
# What is the average reading test score of males in the train set?

pisaTrain.groupby('male')['readingScore'].mean()

male
0    512.940631
1    483.532479
Name: readingScore, dtype: float64

In [8]:
# Which variables are missing data in at least one observation in the training set? 

pisaTrain.isnull().any().sort_values()

grade                    False
urban                    False
publicSchool             False
readingScore             False
male                     False
expectBachelors           True
raceeth                   True
schoolHasLibrary          True
studentsInEnglish         True
minutesPerWeekEnglish     True
read30MinsADay            True
computerForSchoolwork     True
englishAtHome             True
fatherBornUS              True
motherBornUS              True
schoolSize                True
fatherWork                True
fatherBachelors           True
fatherHS                  True
motherWork                True
motherBachelors           True
motherHS                  True
preschool                 True
selfBornUS                True
dtype: bool

In [9]:
# Linear regression discards observations with missing data, 
# so we want to remove all null observations from the 
# training and testing sets.

pisaTrain = pisaTrain.dropna(how='any')
pisaTest = pisaTest.dropna(how='any')

pisaTrain.shape , pisaTest.shape

((2414, 24), (990, 24))

In [10]:
# Examine the first 5 unique values for each variable
# to get a sense of the type of variables in the dataframe

for var in pisaTrain:
    print("{} :: {}".format( var , pisaTrain[var].unique()[:5]) )

grade :: [11 10  9 12  8]
male :: [1 0]
raceeth :: ['White' 'Black' 'Hispanic' 'More than one race'
 'American Indian/Alaska Native']
preschool :: [ 0.  1.]
expectBachelors :: [ 0.  1.]
motherHS :: [ 1.  0.]
motherBachelors :: [ 1.  0.]
motherWork :: [ 1.  0.]
fatherHS :: [ 1.  0.]
fatherBachelors :: [ 0.  1.]
fatherWork :: [ 1.  0.]
selfBornUS :: [ 1.  0.]
motherBornUS :: [ 1.  0.]
fatherBornUS :: [ 1.  0.]
englishAtHome :: [ 1.  0.]
computerForSchoolwork :: [ 1.  0.]
read30MinsADay :: [ 1.  0.]
minutesPerWeekEnglish :: [ 450.  200.  250.  300.  294.]
studentsInEnglish :: [ 25.  23.  35.  30.  24.]
schoolHasLibrary :: [ 1.  0.]
publicSchool :: [1 0]
urban :: [0 1]
schoolSize :: [ 1173.  2640.  1095.  1913.   899.]
readingScore :: [ 575.01  458.11  613.89  439.36  465.9 ]


In [11]:
# Factor variables are variables that take on a 
# discrete set of values, like the "Region" variable
# in the WHO dataset from the second lecture of Unit 1.
# This is an unordered factor because there isn't
# any natural ordering between the levels. An ordered
# factor has a natural ordering between the 
# levels (an example would be the
# classifications "large," "medium," and "small").

# Which of the following variables is an unordered 
# factor with at least 3 levels? (Select all that apply.)

# Which of the following variables is an ordered 
# factor with at least 3 levels? (Select all that apply.)

variables = ['grade','raceeth','male']

for var in variables:
    print("{} :: {}".format(var,pisaTrain[var].unique()))

grade :: [11 10  9 12  8]
raceeth :: ['White' 'Black' 'Hispanic' 'More than one race'
 'American Indian/Alaska Native' 'Asian'
 'Native Hawaiian/Other Pacific Islander']
male :: [1 0]


In [ ]:
# To include unordered factors in a linear regression model, 
# we define one level as the "reference level" and add a binary 
# variable for each of the remaining levels. In this way, a 
# factor with n levels is replaced by n-1 binary variables. 
# The reference level is typically selected to be the most 
# frequently occurring level in the dataset.

In [12]:
# Convert some variables to categorical type:

pisaTrain.raceeth = pisaTrain.raceeth.astype('category',ordered=False)
pisaTest.raceeth = pisaTest.raceeth.astype('category',ordered=False)

In [13]:
# Set 'White to be the first unordered level. 
# This makes statsmodels take it as the reference level during regression.

reordered = ['White', 'Black', 'Hispanic', 
             'More than one race', 'American Indian/Alaska Native', 
             'Asian', 'Native Hawaiian/Other Pacific Islander']

pisaTrain.raceeth = pisaTrain.raceeth.cat.reorder_categories(reordered)
pisaTest.raceeth = pisaTest.raceeth.cat.reorder_categories(reordered)

In [14]:
# Build a linear regression model (call it lmScore) using 
# the training set to predict readingScore using all the 
# remaining variables.

formula = "readingScore ~ {}".format(
          ' + '.join(pisaTrain.columns.difference(['readingScore'])))

lmScore = sm.ols(formula=formula, data=pisaTrain).fit()
print(lmScore.summary2())

                                   Results: Ordinary least squares
Model:                           OLS                         Adj. R-squared:                0.317     
Dependent Variable:              readingScore                AIC:                           27647.0896
Date:                            2016-04-30 23:52            BIC:                           27814.9717
No. Observations:                2414                        Log-Likelihood:                -13795.   
Df Model:                        28                          F-statistic:                   41.04     
Df Residuals:                    2385                        Prob (F-statistic):            1.72e-180 
R-squared:                       0.325                       Scale:                         5448.0    
------------------------------------------------------------------------------------------------------
                                                   Coef.   Std.Err.    t     P>|t|    [0.025   0.975] 
------

In [15]:
# What is the Multiple R-squared value of lmScore on the training set?

lmScore.rsquared

0.32514335592135679

In [16]:
# What is the training-set root-mean squared error (RMSE) of lmScore?
# The RMSE is the square root of the mean/average of the square of the residuals.

np.sqrt(np.mean(lmScore.resid**2))

73.365551432984489

In [17]:
# The RMSE can be computed by first computing the SSE (sum of squared errors)
# and then dividing by the number of observations and taking the square root

residuals = pisaTrain.readingScore - lmScore.fittedvalues

SSE = sum((residuals)**2)

np.sqrt(SSE / len(pisaTrain))

73.365551432984489

In [18]:
# Consider two students A and B. They have all variable 
# values the same, except that: 

# student A is in grade 11 
# student B is in grade 9

# What is the predicted reading score of student A 
# minus the predicted reading score of student B?

lmScore.params.grade*2

59.085414183537857

In [19]:
# Based on the significance codes, which variables are candidates 
# for removal from the model (ha a p-value of over 0.1)?

(lmScore.pvalues > 0.1).sort_values(ascending=False)

urban                                                 True
englishAtHome                                         True
raceeth[T.Native Hawaiian/Other Pacific Islander]     True
fatherBornUS                                          True
fatherHS                                              True
fatherWork                                            True
studentsInEnglish                                     True
raceeth[T.Asian]                                      True
minutesPerWeekEnglish                                 True
motherBornUS                                          True
motherHS                                              True
motherWork                                            True
preschool                                             True
schoolHasLibrary                                      True
selfBornUS                                            True
raceeth[T.American Indian/Alaska Native]             False
raceeth[T.More than one race]                        Fal

In [20]:
# Use the lmScore model to predict the reading scores of students in pisaTest.
# Call this vector of predictions "predictions". 
# Do not change the variables in the model

predictions = lmScore.predict(pisaTest)

In [32]:
predictions

array([ 471.98671346,  540.19737943,  429.75234341,  499.01386793,
        564.0212217 ,  517.90643536,  530.59309175,  408.55455704,
        538.93975137,  536.27439872,  477.83398767,  403.2319115 ,
        460.33383127,  496.92554017,  519.09527878,  557.28988482,
        488.29472309,  498.29986405,  566.97619025,  538.20018708,
        469.2967337 ,  539.90457871,  535.55697514,  476.15578855,
        534.80173938,  547.55445453,  530.70667433,  530.39919138,
        556.29853323,  426.51490202,  521.81992341,  483.05400117,
        480.24115336,  467.72531559,  486.95967332,  536.98298324,
        564.17064087,  495.00575631,  429.48959834,  492.95823492,
        519.98624465,  570.2958081 ,  547.74989888,  592.41829094,
        486.24367309,  587.08907378,  523.39199575,  559.24048414,
        541.30700228,  522.20525584,  511.34136692,  505.33886352,
        528.27494246,  471.01430153,  504.87825532,  414.94188636,
        519.77491862,  528.36160264,  541.81857274,  522.96903

In [21]:
# What is the range between the maximum and minimum predicted reading score on the test set?

predictions.max() - predictions.min()

284.46831179513026

In [22]:
# What is the sum of squared errors (SSE) of lmScore on the testing set?

residuals = pisaTest.readingScore - predictions

SSE = sum(residuals**2)
SSE

5762082.3711444354

In [23]:
# What is the root-mean squared error (RMSE) of lmScore on the testing set?

# The RMSE can be computed by first computing the SSE (sum of squared errors)
# and then dividing by the number of observations and taking the square root

np.sqrt(np.mean(SSE / len(pisaTest)))

76.290793831092216

In [24]:
# What is the predicted test score used in the baseline model? 
# Remember to compute this value using the training set and not the test set.

formula = "readingScore ~ 1"

lmScore_baseline = sm.ols(formula=formula, data=pisaTrain).fit()
lmScore_baseline.params

Intercept    517.962887
dtype: float64

In [25]:
# Another way:

np.mean(pisaTrain.readingScore)

517.9628873239429

In [26]:
# What is the sum of squared errors of the baseline model 
# on the testing set? HINT: We call the sum of squared errors 
# for the baseline model the total sum of squares (SST).

predictions_baseline = lmScore_baseline.predict(pisaTest)

SST = sum((pisaTest.readingScore - predictions_baseline)**2)
SST

7802354.0776138371

In [59]:
# Another way:

sum( (pisaTest.readingScore - np.mean(pisaTrain.readingScore))**2)

7802354.0776138389

In [60]:
# What is the test-set R-squared value of lmScore?

SSE = sum((pisaTest.readingScore - predictions)**2)
SST = sum((pisaTest.readingScore - np.mean(pisaTrain.readingScore))**2)
test_set_r_squared = 1 - (SSE/SST)

SSE,SST,test_set_r_squared

(5762082.3711444354, 7802354.0776138389, 0.2614943754376976)

In [53]:
def structure(df):
    
    print("{0} obs. of {1} variables".format(df.shape[0],df.shape[1]))
    for variable in df:
        print("$ {}: {}".format(variable,df[variable].unique()))


2414 obs. of 24 variables
$ grade: [11 10  9 12  8]
$ male: [1 0]
$ raceeth: [White, Black, Hispanic, More than one race, American Indian/Alaska Native, Asian, Native Hawaiian/Other Pacific Islander]
Categories (7, object): [White, Black, Hispanic, More than one race, American Indian/Alaska Native, Asian, Native Hawaiian/Other Pacific Islander]
$ preschool: [ 0.  1.]
$ expectBachelors: [ 0.  1.]
$ motherHS: [ 1.  0.]
$ motherBachelors: [ 1.  0.]
$ motherWork: [ 1.  0.]
$ fatherHS: [ 1.  0.]
$ fatherBachelors: [ 0.  1.]
$ fatherWork: [ 1.  0.]
$ selfBornUS: [ 1.  0.]
$ motherBornUS: [ 1.  0.]
$ fatherBornUS: [ 1.  0.]
$ englishAtHome: [ 1.  0.]
$ computerForSchoolwork: [ 1.  0.]
$ read30MinsADay: [ 1.  0.]
$ minutesPerWeekEnglish: [  450.   200.   250.   300.   294.   232.   225.   270.   275.   315.
   235.   600.   240.  1296.    90.   180.   290.   460.    70.    40.
   205.    50.   190.   360.     0.   344.   285.   550.   420.   160.
   260.   354.   255.    84.   264.    55.   25